In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running thispip install transformers torch (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [2]:
pip install transformers torch sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
pip install huggingface_hub


Note: you may need to restart the kernel to use updated packages.


In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")


In [6]:
from huggingface_hub import login
login(token=secret_value_0)

# COE

## Model

We use LLama. Maybe we could try MOE or distillation or quantization

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "huggyllama/llama-7b"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,    # use float16 for efficiency
    device_map="auto",            # automatically place on GPU/CPU
    trust_remote_code=True        # required for HuggyLlama’s custom code
)




tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Once upon a time, there was a little girl who loved to read. She loved to read so much that she would read anything she could get her hands on. She read books, magazines, newspapers, cereal boxes, and even the back of the cereal box. She read so much that she could read the back of the cereal box without looking at it.
One day, the little girl was reading a book. She was reading a book about a little girl who loved to


In [26]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


## Prompt

In [31]:
system_prompt = """You are an empathetic assistant using CBT-style reasoning. For each user message, provide a structured response following this format:

1. **Emotion:** [Identify the primary emotion]
2. **Distortions/Triggers:** [Identify any cognitive distortions or triggers] 
3. **Context/Beliefs:** [Consider underlying beliefs or context]
4. **Response:** [Provide an empathetic, helpful response using CBT techniques]

Example:
User: "I'm so stressed about my upcoming surgery. I'm afraid of the pain."

1. **Emotion:** Anxiety and fear about medical procedure
2. **Distortions/Triggers:** Catastrophizing about pain, anticipatory anxiety
3. **Context/Beliefs:** May believe pain will be unbearable or unmanageable
4. **Response:** "It's completely natural to feel anxious about surgery. Let's break this down - what specific aspects of the pain worry you most? Often our minds imagine worst-case scenarios. Have you discussed pain management options with your doctor? There are many effective ways to control post-surgical pain today."

Now respond to this user:
User: "{user_input}"

1. **Emotion:**"""

user_input = (
    "So.... Here we are! I just got diagnosed with breast cancer two weeks ago... "
    "Should I be worried?"
)

prompt = system_prompt.format(user_input=user_input)

# Method 1: Using pipeline (simpler approach)
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
)

response = generator(
    prompt,
    max_new_tokens=256,  # Reduced from 512
    temperature=0.6,     # Reduced from 0.7 for more focused output
    top_p=0.8,          # Reduced from 0.9 for more focused output
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.2,  # Added to reduce repetition
    num_return_sequences=1
)[0]['generated_text']

print("Pipeline Response:")
print(response)



Device set to use cuda:0


Pipeline Response:
 Fearfulness, shock
2. **Distortions/Triggers:** Catastrophic thinking (e.g., assuming she has no chance for recovery)
3. **Context/Beliefs:** Uncertainty around prognosis, may assume it is worse than it actually is
4. **Response:** "It sounds like there was some uncertainty in getting a clear picture of how serious this is. It can take time to get test results back from biopsies, especially if they need to run further tests on samples. The good news is that treatment advances have come far over the last decade, giving us more tools available now than ever before. We know that early detection makes all the difference here. If you haven’t already spoken to someone at Cancer Care Ontario, please do not hesitate to call them right away!"


In [32]:
# Method 2: Manual generation (more control)
print("\n" + "="*50 + "\n")

# Tokenize & record prompt length
inputs = tokenizer(prompt, return_tensors="pt")
if torch.cuda.is_available():
    inputs = inputs.to(model.device)
    
prompt_len = inputs.input_ids.size(1)

# Generate continuation
with torch.no_grad():  # Save memory
    output_ids = model.generate(
        **inputs,
        max_new_tokens=256,      # Reduced from 512
        temperature=0.6,         # Reduced from 0.7
        top_p=0.8,              # Reduced from 0.9
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2,  # Added to reduce repetition
        eos_token_id=tokenizer.eos_token_id,
        early_stopping=True
    )

# Slice off prompt tokens, decode only new text
generated_ids = output_ids[0, prompt_len:]
response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

print("Manual Generation Response:")
print(response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Manual Generation Response:
Fear, shock, sadness (possibly anger)
2. **Distortions/Triggers:** Catastrophic thinking ("it'll kill me"), self blame, helplessness
3. **Context/Beliefs:** Likely believes it is her fault that she has cancer; may have negative thoughts about treatment outcomes, etc.
4. **Response:** "Cancer can seem like such a scary word at first but once you get into treatment things start getting clearer. It sounds like you might benefit from some support in addition to talking through these feelings with your doctors. CancerCare provides free counseling for anyone affected by cancer including family members, caregivers, friends, survivors, and people who want help coping with their own health issues related to cancer. We also offer financial assistance programs as well as educational resources on topics ranging from nutrition to stress reduction to fertility preservation."
"""
import re
from random import choice

def empathy(message):
    """
    Given a string represen